original: [openai-cookbook/examples/How_to_finetune_chat_models.ipynb at main · openai/openai-cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb)

# How to fine-tune chat models

このノートブックは、新しいgpt-3.5-turboのファインチューニングのためのガイドを提供します。

[RecipeNLG](https://github.com/Glorf/recipenlg)データセットを使用し、エンティティ抽出を実行します。

このデータセットは、様々なレシピと、それぞれから抽出された一般的な成分のリストを提供します。これは、固有表現認識（NER）タスクに共通のデータセットです。

次の手順を実行します。

1. セットアップ：データセットをロードし、ファインチューニングする1つのドメインに絞り込みます。
2. データの準備：トレーニングと検証のサンプルを作成し、それらをファイルエンドポイントにアップロードすることで、ファインチューニング用のデータを準備します。
3. ファインチューニング：ファインチューニングモデルを作成します。
4. 推論：新しい入力の推論に微調整されたモデルを使用します。

これが終わるまでに、ファインチューニングされたgpt-3.5-turboモデルをトレーニング、評価、デプロイできるようになります。

ファインチューニングの詳細：

- [Fine-tuning - OpenAI API](https://platform.openai.com/docs/guides/fine-tuning)
- [API Reference - OpenAI API](https://platform.openai.com/docs/api-reference/fine-tuning)
- [GPT-3.5 Turbo fine-tuning and API updates](https://openai.com/blog/gpt-3-5-turbo-fine-tuning-and-api-updates)

## Setup

In [1]:
# openai pythonパッケージの最新バージョンを必ず使用する
%pip install -U openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import json
import openai
import os
import pandas as pd
from pprint import pprint

ファインチューニングは、特定のドメインに焦点を当てた場合に最適に機能します。データセットがモデルが学習できるように十分に焦点が絞られていると同時に、まだ見たことのないサンプルが見逃されない程度に一般的であることを確認することが重要です。

これを念頭に置いて、 www.cookbooks.com のドキュメントのみを含むようにRecipeNLGデータセットからサブセットを抽出しました。

In [4]:
# このタスクに使用するデータセットを読み込みます
# これはRecipeNLGデータセットになります。 www.cookbooks.comのドキュメントのみを含むようにクリーンアップしました。
recipe_df = pd.read_csv("./data/cookbook_recipes_nlg_10k.csv")
recipe_df.head()

,title,ingredients,directions,link,source,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,www.cookbooks.com,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,www.cookbooks.com,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,www.cookbooks.com,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,www.cookbooks.com,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,www.cookbooks.com,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


## データの準備

データを準備するところから始めます。

ChatCompletion形式でファインチューニングする場合、各トレーニングサンプルはメッセージの単純なリストになります。たとえば、エントリは次のようになります。

```
[{'role': 'system',
  'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'},
```

```
 {'role': 'user',
  'content': 'Title: No-Bake Nut Cookies\n\nIngredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]\n\nGeneric ingredients: '},
```

```
 {'role': 'assistant',
  'content': '["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]'}]
```

トレーニングプロセス中、この会話は分割され、最後のエントリはモデルが生成する完了であり、残りのメッセージはプロンプトとして機能します。

トレーニングサンプルを作成するときは、これを考慮してください。モデルが複数ターンの会話で動作する場合は、会話が拡大し始めたときにパフォーマンスが低下しないように、代表的なサンプルを提供してください。

現在、各トレーニングサンプルには4096トークンの制限があることに注意してください。これより長いものは4096トークンで切り捨てられます。

In [18]:
training_data = []

system_message = "You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided."

def prepare_example_conversation(row):
    messages = []
    messages.append({"role": "system", "content": system_message})

    user_message = f"""Title: {row['title']}\n\nIngredients: {row['ingredients']}\n\nGeneric ingredients: """
    messages.append({"role": "user", "content": user_message})

    messages.append({"role": "assistant", "content": row["NER"]})

    return {"messages": messages}

pprint(prepare_example_conversation(recipe_df.iloc[0]))

{'messages': [{'content': 'You are a helpful recipe assistant. You are to '
                          'extract the generic ingredients from each of the '
                          'recipes provided.',
               'role': 'system'},
              {'content': 'Title: No-Bake Nut Cookies\n'
                          '\n'
                          'Ingredients: ["1 c. firmly packed brown sugar", '
                          '"1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 '
                          'c. broken nuts (pecans)", "2 Tbsp. butter or '
                          'margarine", "3 1/2 c. bite size shredded rice '
                          'biscuits"]\n'
                          '\n'
                          'Generic ingredients: ',
               'role': 'user'},
              {'content': '["brown sugar", "milk", "vanilla", "nuts", '
                          '"butter", "bite size shredded rice biscuits"]',
               'role': 'assistant'}]}


次に、トレーニングデータとして使用するデータセットのサブセットに対してこれを実行してみましょう。

30~50この適切に剪定された例から始めることもできます。トレーニングセットのサイズを増やすと、パフォーマンスが直線的に拡大し続けることがわかりますが、ジョブの時間も長くなります。

In [19]:
# データセットの最初の100行をトレーニングに使用します
training_df = recipe_df.loc[0:100]

# prepare_example_conversation関数をtraining_dfの各行に適用します
training_data = training_df.apply(prepare_example_conversation, axis=1).to_list()

for example in training_data[:5]:
    print(example)

{'messages': [{'role': 'system', 'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'}, {'role': 'user', 'content': 'Title: No-Bake Nut Cookies\n\nIngredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]\n\nGeneric ingredients: '}, {'role': 'assistant', 'content': '["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]'}]}
{'messages': [{'role': 'system', 'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'}, {'role': 'user', 'content': 'Title: Jewell Ball\'S Chicken\n\nIngredients: ["1 small jar chipped beef, cut up", "4 boned chicken breasts", "1 can cream of mushroom soup", "1 carton sour cream"]\n\nGeneric ingredients: '}, {'role': 'assistant', 'content': '["bee

トレーニングデータに加えて、モデルがトレーニングセットに過剰適合していないことを確認するために使用される検証データもオプションで提供できます。

In [20]:
validation_df = recipe_df.loc[101:200]
validation_data = validation_df.apply(prepare_example_conversation, axis=1).tolist()

次に、データを`.jsonl`ファイルとして保存する必要があります。各行は1つのトレーニング会話例です。

In [21]:
def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)

In [22]:
training_file_name = "tmp_recipe_finetune_training.jsonl"
write_jsonl(training_data, training_file_name)

validation_file_name = "tmp_recipe_finetune_validation.jsonl"
write_jsonl(validation_data, validation_file_name)

トレーニング`.jsonl`ファイルの最初の5行は次のようになります。

In [23]:
# トレーニングファイルの最初の5行を出力します
# for linux
# !head -n 5 tmp_recipe_finetune_training.jsonl
# for windows (powershell)
!Powershell.exe -Command "type .\tmp_recipe_finetune_training.jsonl -Head 5"

{"messages": [{"role": "system", "content": "You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided."}, {"role": "user", "content": "Title: No-Bake Nut Cookies\n\nIngredients: [\"1 c. firmly packed brown sugar\", \"1/2 c. evaporated milk\", \"1/2 tsp. vanilla\", \"1/2 c. broken nuts (pecans)\", \"2 Tbsp. butter or margarine\", \"3 1/2 c. bite size shredded rice biscuits\"]\n\nGeneric ingredients: "}, {"role": "assistant", "content": "[\"brown sugar\", \"milk\", \"vanilla\", \"nuts\", \"butter\", \"bite size shredded rice biscuits\"]"}]}
{"messages": [{"role": "system", "content": "You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided."}, {"role": "user", "content": "Title: Jewell Ball'S Chicken\n\nIngredients: [\"1 small jar chipped beef, cut up\", \"4 boned chicken breasts\", \"1 can cream of mushroom soup\", \"1 carton sour cream\"]\n\nGeneric ingredients: "}, {"role":

## ファイルのアップロード

これで、ファイルをファイルエンドポイントにアップロードして、微調整されたモデルで使用できるようになりました。

In [26]:
training_response = openai.File.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response["id"]

validation_response = openai.File.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response["id"]

print("Training file ID: ", training_file_id)
print("Vaidation file ID: ", validation_file_id)

Training file ID:  file-OCWzCknyiRyzN0Y0FwPFehWv
Vaidation file ID:  file-ZO9wdLuzneUDkVouldsvQeG2


## ファインチューニング

これで、生成されたファイルとモデルを識別するためのオプションのサフィックスを使用して、ファインチューニングジョブを作成できます。応答には、ジョブの更新を取得するために使用できるIDが含まれます。

注：ファイルは最初にシステムによって処理される必要があるため、「ファイルの準備ができていません」エラーが発生する可能性があります。その場合は、数分後に再試行してください。

In [27]:
response = openai.FineTuningJob.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-3.5-turbo",
    suffix="recipe-ner"
)

job_id = response["id"]

print("Job ID:", response["id"])
print("Status:", response["status"])

Job ID: ftjob-Yjrf3FgY0XhYavCXO8N44wed
Status: created


## ジョブのステータスをチェック

https://api.openai.com/v1/alpha/fine-tunes エンドポイントに`GET`リクエストを送信して、アルファファインチューニングジョブを一覧表示できます。この例では、前の手順で取得したIDが`status: succeeded`になることを確認する必要があります。

完了したら、`result_files`を使用して検証セットから結果をサンプリングし（検証セットをアップロードした場合）、`fine_tuned_model`パラメーターのIDを使用してトレーニング済モデルを呼び出すことができます。

In [28]:
response = openai.FineTuningJob.retrieve(job_id)

print("Job ID:", response["id"])
print("Status:", response["status"])
print("Trained Tokens:", response["trained_tokens"])

Job ID: ftjob-Yjrf3FgY0XhYavCXO8N44wed
Status: running
Trained Tokens: None


イベントエンドポイントを使用してファインチューニングの進行状況を追跡できます。ファインチューニングの準備が完了するまで、以下のセルを数回再実行できます。

In [38]:
response = openai.FineTuningJob.list_events(id=job_id, limit=50)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])

Created fine-tune: ftjob-Yjrf3FgY0XhYavCXO8N44wed
Fine tuning job started
Step 10: training loss=0.07
Step 20: training loss=0.01
Step 30: training loss=0.10
Step 40: training loss=0.00
Step 50: training loss=0.03
Step 60: training loss=0.15
Step 70: training loss=0.18
Step 80: training loss=0.53
Step 90: training loss=0.01
Step 100: training loss=0.05
Step 110: training loss=0.07
Step 120: training loss=0.00
Step 130: training loss=0.07
Step 140: training loss=0.00
Step 150: training loss=0.00
Step 160: training loss=0.30
Step 170: training loss=0.80
Step 180: training loss=0.00
Step 190: training loss=0.00
Step 200: training loss=0.21
Step 210: training loss=0.00
Step 220: training loss=0.45
Step 230: training loss=1.18
Step 240: training loss=0.00
Step 250: training loss=0.00
Step 260: training loss=0.26
Step 270: training loss=0.00
Step 280: training loss=0.00
Step 290: training loss=0.00
Step 300: training loss=0.00
New fine-tuned model created: ft:gpt-3.5-turbo-0613:ijiwarunahell

これで、ジョブからファインチューニングされたモデルIDを取得できるようになります。

In [39]:
response = openai.FineTuningJob.retrieve(job_id)
fine_tuned_model_id = response["fine_tuned_model"]

print("Fine-tuned model ID:", fine_tuned_model_id)

Fine-tuned model ID: ft:gpt-3.5-turbo-0613:ijiwarunahello:recipe-ner:7qsfhZfd


## 推論

最後のステップは、ファインチューニングしたモデルを推論に使用することです。従来の`FineTuning`と同様に、新しいファインチューニングされたモデル名をモデルパラメータに入力して`ChatCompletions`を呼び出すだけです。

In [44]:
print(training_data[-1]["messages"][0])

{'role': 'system', 'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'}


In [45]:
# cookbookの実装では動かない（2023.08.24現在）
# test_row = test_df.iloc[0]
# test_messages = []
# test_messages.append({"role": "system", "content": system_message})
# user_message = create_user_message(test_row)
# test_messages.append({"role": "user", "content": create_user_message(test_row)})

# 以下で対処
def create_user_message(data):
    user_message = {}
    messages = data["messages"]
    for message in messages:
        if message.get("role", None) == "user":
            user_message = message
            break
    return user_message

test_data = training_data[-1]
test_messages = []
test_messages.append({"role": "system", "content": system_message})
user_message = create_user_message(test_data)
test_messages.append(user_message)

pprint(test_messages)

[{'content': 'You are a helpful recipe assistant. You are to extract the '
             'generic ingredients from each of the recipes provided.',
  'role': 'system'},
 {'content': 'Title: Pancakes\n'
             '\n'
             'Ingredients: ["1 c. flour", "1 tsp. soda", "1 tsp. salt", "1 '
             'Tbsp. sugar", "1 egg", "3 Tbsp. margarine, melted", "1 c. '
             'buttermilk"]\n'
             '\n'
             'Generic ingredients: ',
  'role': 'user'}]


In [46]:
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=500
)
print(response["choices"][0]["message"]["content"])

["flour", "soda", "salt", "sugar", "egg", "margarine", "buttermilk"]


## オプション：データフォーマットの確認

参考：[Fine-tuning - OpenAI API](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset)

データセットをコンパイルし、ファインチューニングジョブを作成する前に、データのフォーマットをチェックすることが重要です。

- 潜在的なエラーの発見
- トークン数の確認
- ファインチューニングジョブのコストの見積もり

In [47]:
%pip install tiktoken

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [49]:
import json
import os
import tiktoken
import numpy as np
from collections import defaultdict


def check_data_formatting(data):
    # データフォーマットのチェック
    # 例数と最初の項目をチェックすることで、データを素早く検査することができる
    print("Num examples: ", len(data))
    print("First example:")
    for message in data[0]["messages"]:
        print(message)

    # データの感覚を掴んだところで、すべての異なる例を見て、書式が正しいか、チャットの完了メッセージの構造と一致しているかをチェックする必要があります。

    # フォーマットエラーのチェック
    format_errors = defaultdict(int)
    for ex in data:
        if not isinstance(ex, dict):
            format_errors["data_type"] += 1
            continue

        messages = ex.get("messages", None)
        if not messages:
            format_errors["missing_message_list"] += 1
            continue

        for message in messages:
            if "role" not in message or "content" not in message:
                format_errors["message_missing_key"] += 1

            if any(k not in ("role", "content", "name") for k in message):
                format_errors["message_unrecognized_key"] += 1

            if message.get("role", None) not in ("system", "user", "assistant"):
                format_errors["unrecognized_role"] += 1

            content = message.get("content", None)
            if not content or not isinstance(content, str):
                format_errors["missing_content"] += 1

        if not any(message.get("role", None) == "assistant" for message in messages):
            format_errors["example_missing_assistant_message"] += 1

    if format_errors:
        print("Found errors:")
        for k, v in format_errors.items():
            print(f"{k}: {v}")
    else:
        print("No errors found")

In [52]:
# メッセージの構造だけでなく、長さが4096トークンの制限を超えないようにする必要もあります。

# トークンカウント関数
encoding = tiktoken.get_encoding("cl100k_base")

# 正確ではないことに注意する
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

MAX_TOKENS_PER_EXAMPLE = 4096
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
TARGET_EPOCHS = 3
MIN_EPOCHS = 1
MAX_EPOCHS = 25

def warnings_and_tokens_count(dataset):
    n_missing_system = 0
    n_missing_user = 0
    n_messages = []
    convo_lens = []
    assistant_message_lens = []

    for ex in dataset:
        messages = ex["messages"]
        if not any(message["role"] == "system" for message in messages):
            n_missing_system += 1
        if not any(message["role"] == "user" for message in messages):
            n_missing_user += 1
        n_messages.append(len(messages))
        convo_lens.append(num_tokens_from_messages(messages))
        assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

    print("Num examples missing system message:", n_missing_system)
    print("Num examples missing user message:", n_missing_user)
    print_distribution(n_messages, "num_messages_per_example")
    print_distribution(convo_lens, "num_total_tokens_per_example")
    print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
    n_too_long = sum(l > 4096 for l in convo_lens)
    print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")


    # pricing and default n_epochs estimate
    n_epochs = TARGET_EPOCHS
    n_train_examples = len(dataset)
    if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
        n_epochs = min(MAX_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
    elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
        n_epochs = max(MIN_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

    n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
    print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
    print(f"By default, you'll train for {n_epochs} epochs on this dataset")
    print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")
    print("See pricing page to estimate total costs")

In [53]:
check_data_formatting(training_data)
warnings_and_tokens_count(training_data)

Num examples:  101
First example:
{'role': 'system', 'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'}
{'role': 'user', 'content': 'Title: No-Bake Nut Cookies\n\nIngredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]\n\nGeneric ingredients: '}
{'role': 'assistant', 'content': '["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]'}
No errors found
Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 69, 227
mean / median: 134.16831683168317, 130.0
p5 / p95: 97.0, 180.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 8, 58
m

In [54]:
check_data_formatting(validation_data)
warnings_and_tokens_count(validation_data)

Num examples:  100
First example:
{'role': 'system', 'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'}
{'role': 'user', 'content': 'Title: Crustless Vegetable Ham Pie\n\nIngredients: ["1/4 c. butter", "1/4 lb. mushrooms, sliced", "1 garlic clove, minced", "1 medium zucchini, sliced thinly", "1/4 c. chopped onion", "1 c. diced, cooked ham", "4 eggs", "2 c. Ricotta cheese", "1 c. shredded Monterey Jack", "1 (10 oz.) pkg. frozen spinach, thawed and drained", "1/2 tsp. dill weed", "salt and pepper"]\n\nGeneric ingredients: '}
{'role': 'assistant', 'content': '["butter", "mushrooms", "garlic", "zucchini", "onion", "eggs", "Ricotta cheese", "shredded Monterey Jack", "frozen spinach", "dill weed", "salt"]'}
No errors found
Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distrib

# 参考情報

- [固有表現抽出（NER）とはどんなタスクか【自然言語処理タスク紹介1】 | もじとばコム](https://mojitoba.com/2021/03/03/what_is_ner/)
- [【python】jsonっぽいファイル「jsonl（JSON LINES）」をcsvに書き換えるまで｜じゅにこ](https://note.com/junico02_se_blog/n/n5ed9c474aed4)
- [RecipeNLG](https://recipenlg.cs.put.poznan.pl/)
